In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def import_csv_to_mysql():
    """CSV 데이터를 MySQL hospitals 테이블에 import"""
    
    print("="*60)
    print("CSV 데이터 MySQL Import 시작")
    print("="*60)
    
    # 1. CSV 파일 읽기
    print("\n[1단계] CSV 파일 로드 중...")
    csv_path = '../data/processed/busan_hospitals_static.csv'
    
    try:
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
        print(f"✓ CSV 파일 로드 성공: {len(df)}개 레코드")
    except Exception as e:
        print(f"✗ CSV 파일 로드 실패: {e}")
        return
    
    # 2. MySQL 연결
    print("\n[2단계] MySQL 데이터베이스 연결 중...")
    
    try:
        connection = mysql.connector.connect(
            host='localhost',        # MySQL 서버 주소
            port=3306,              # MySQL 포트
            database='emergency_room',  # 데이터베이스 이름
            user='root',            # MySQL 사용자명 (본인 것으로 변경)
            password='1234'  # MySQL 비밀번호 (본인 것으로 변경 필요!)
        )
        
        if connection.is_connected():
            print("✓ MySQL 연결 성공")
            cursor = connection.cursor()
            
            # 3. 기존 데이터 삭제 (선택사항)
            print("\n[3단계] 기존 데이터 확인 중...")
            cursor.execute("SELECT COUNT(*) FROM hospitals")
            existing_count = cursor.fetchone()[0]
            
            if existing_count > 0:
                print(f"  기존 데이터 {existing_count}개 발견")
                user_input = input("  기존 데이터를 삭제하고 새로 입력하시겠습니까? (y/n): ")
                
                if user_input.lower() == 'y':
                    cursor.execute("DELETE FROM hospitals")
                    connection.commit()
                    print("  ✓ 기존 데이터 삭제 완료")
                else:
                    print("  기존 데이터 유지. 중복 데이터가 생길 수 있습니다.")
            
            # 4. 데이터 삽입
            print("\n[4단계] 데이터 삽입 중...")
            
            insert_query = """
            INSERT INTO hospitals (
                id, name, address, phone, emergency_phone,
                latitude, longitude, district, emergency_level,
                hospital_type, beds_total, has_ct, has_mri
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
            
            success_count = 0
            error_count = 0
            
            for index, row in df.iterrows():
                try:
                    data = (
                        row['id'],
                        row['name'],
                        row['address'],
                        row['phone'],
                        row['emergency_phone'] if pd.notna(row['emergency_phone']) else None,
                        float(row['latitude']),
                        float(row['longitude']),
                        row['district'],
                        row['emergency_level'],
                        row['hospital_type'],
                        int(row['beds_total']) if pd.notna(row['beds_total']) else None,
                        bool(row['has_ct']) if pd.notna(row['has_ct']) else False,
                        bool(row['has_mri']) if pd.notna(row['has_mri']) else False
                    )
                    
                    cursor.execute(insert_query, data)
                    success_count += 1
                    
                    if (index + 1) % 10 == 0:
                        print(f"  진행 중... {index + 1}/{len(df)}")
                
                except Error as e:
                    error_count += 1
                    print(f"  ✗ 오류 발생 (행 {index + 1}): {e}")
            
            # 5. 커밋
            connection.commit()
            
            # 6. 결과 확인
            print("\n[5단계] Import 결과 확인...")
            cursor.execute("SELECT COUNT(*) FROM hospitals")
            final_count = cursor.fetchone()[0]
            
            print("\n" + "="*60)
            print("✓ CSV 데이터 MySQL Import 완료!")
            print("="*60)
            print(f"성공: {success_count}개")
            print(f"실패: {error_count}개")
            print(f"최종 DB 레코드 수: {final_count}개")
            
            # 7. 샘플 데이터 확인
            print("\n[샘플 데이터 확인]")
            cursor.execute("SELECT id, name, district, emergency_level FROM hospitals LIMIT 5")
            results = cursor.fetchall()
            
            print("\nID     | 병원명            | 구       | 등급")
            print("-" * 60)
            for row in results:
                print(f"{row[0]} | {row[1]:<15} | {row[2]:<8} | {row[3]}")
            
    except Error as e:
        print(f"✗ MySQL 연결 오류: {e}")
        print("\n확인사항:")
        print("1. MySQL 서버가 실행 중인가요?")
        print("2. 데이터베이스 이름이 'emergency_room'인가요?")
        print("3. 사용자명과 비밀번호가 올바른가요?")
        return
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("\n✓ MySQL 연결 종료")

if __name__ == "__main__":
    # MySQL 비밀번호 확인 필요!
    print("⚠️  스크립트 실행 전에 MySQL 비밀번호를 코드에서 수정하세요!")
    print("   line 21: password='your_password' ← 본인 비밀번호로 변경\n")
    
    proceed = input("비밀번호를 수정했습니까? (y/n): ")
    
    if proceed.lower() == 'y':
        import_csv_to_mysql()
    else:
        print("스크립트를 수정 후 다시 실행하세요.")

⚠️  스크립트 실행 전에 MySQL 비밀번호를 코드에서 수정하세요!
   line 21: password='your_password' ← 본인 비밀번호로 변경



비밀번호를 수정했습니까? (y/n):  y


CSV 데이터 MySQL Import 시작

[1단계] CSV 파일 로드 중...
✓ CSV 파일 로드 성공: 46개 레코드

[2단계] MySQL 데이터베이스 연결 중...
✓ MySQL 연결 성공

[3단계] 기존 데이터 확인 중...

[4단계] 데이터 삽입 중...
  진행 중... 10/46
  진행 중... 20/46
  진행 중... 30/46
  진행 중... 40/46

[5단계] Import 결과 확인...

✓ CSV 데이터 MySQL Import 완료!
성공: 46개
실패: 0개
최종 DB 레코드 수: 46개

[샘플 데이터 확인]

ID     | 병원명            | 구       | 등급
------------------------------------------------------------
ER001 | 고신대학교복음병원       | 서구       | 지역응급의료기관
ER002 | 인제대학교부산백병원      | 부산진구     | 지역응급의료기관
ER003 | 부산대학교병원         | 서구       | 지역응급의료기관
ER004 | 동아대학교병원         | 서구       | 지역응급의료기관
ER005 | 학교법인 춘해병원       | 부산진구     | 지역응급의료기관

✓ MySQL 연결 종료
